In [1]:
import cv2
# ROS packages
import numpy as np
import rospy
import geometry_msgs.msg
import time
import roslib; roslib.load_manifest('ur_driver')
import actionlib
# ROS Image message
from sensor_msgs.msg import Image
from std_msgs.msg import String
from thin_obj_bin_picking.msg import img_status
# ROS Image message -> OpenCV2 image converter
from cv_bridge import CvBridge, CvBridgeError
#import tf

import numpy
import sys
import copy
import moveit_commander
import moveit_msgs.msg
import tf
import math
from sensor_msgs.msg import JointState
from math import pi, sin, cos, atan2
from moveit_msgs.msg import RobotTrajectory
from trajectory_msgs.msg import JointTrajectoryPoint
from geometry_msgs.msg import PoseStamped, Pose
from math import sqrt, pi, acos, sin, cos
from robotiq_force_torque_sensor.msg import ft_sensor
from robotiq_c_model_control.msg import _CModel_robot_output as outputMsg
from thin_obj_bin_picking.msg import blister_pose
import re

from geometry_msgs.msg import PoseWithCovariance

In [2]:
import urx
import math3d as m3d
import numpy as np
from math import pi,cos,sin
from numpy.linalg import inv
import rospy
import tf
from robotiq_ft_sensor.msg import ft_sensor
from robotiq_2f_gripper_control.msg import _Robotiq2FGripper_robot_output as outputMsg
from robotiq_2f_gripper_control.msg import _Robotiq2FGripper_robot_input  as inputMsg
import logging
import time
from std_msgs.msg import Int32

In [4]:
john_pub = rospy.Publisher('JohnGripperRobotOutput', Int32, queue_size=10)
rospy.init_node('wide_range_obj_picking',
                  anonymous=True)

logging.basicConfig(level=logging.WARN)

robHong = urx.Robot("192.168.1.102")
robKong = urx.Robot("192.168.1.10")
gripper_pub = rospy.Publisher('Robotiq2FGripperRobotOutput', outputMsg.Robotiq2FGripper_robot_output, queue_size=10)

In [5]:
#global speed for motion of robots
global_speed = 3

In [6]:
#tag detection trigger
is_send_tag = 0

In [7]:
def gactive():
    command = outputMsg.Robotiq2FGripper_robot_output();
    command.rACT = 1
    command.rGTO = 1
    command.rSP  = 255
    command.rFR  = 150					##force need to be adjusted later
    gripper_pub.publish(command)
    rospy.sleep(0.5)
    return command

###___Reset gripper___###
def greset():
    command = outputMsg.Robotiq2FGripper_robot_output();
    command.rACT = 0
    gripper_pub.publish(command)
    rospy.sleep(0.5)

###___Set position of gripper___###
def gposition(degree):
    command = outputMsg.Robotiq2FGripper_robot_output();
    command.rACT = 1
    command.rGTO = 1
    command.rATR = 0
    command.rPR = degree
    command.rSP  = 150
    command.rFR  = 150 ##force need to be adjusted later
    gripper_pub.publish(command)

def jposition(degree):
    john_pub.publish(degree)
    rospy.sleep(0.5)

def go_to_home():
    Hong_joint0 = -pi*46.5/180
    Hong_joint1 = -pi*131/180#-pi*68.03/180 #81.05
    Hong_joint2 = -pi*63.38/180#-pi*137.81/180 #128.91
    Hong_joint3 = -pi/2-(Hong_joint1+Hong_joint2)#-pi*3/2-(group_variable_values[1]+group_variable_values[2])#-pi/2-(group_variable_values[1]+group_variable_values[2])
    Hong_joint4 = -pi*89.16/180#pi*1/2#-pi*1/2
    Hong_joint5 = -pi*1.5/180

    robHong.movej((Hong_joint0,Hong_joint1, Hong_joint2, Hong_joint3, Hong_joint4, Hong_joint5), 1, 0.4)

    Kong_joint0 = -Hong_joint0
    Kong_joint1 = -pi-Hong_joint1
    Kong_joint2 = -Hong_joint2
    Kong_joint3 = -Hong_joint3+pi
    Kong_joint4 = -Hong_joint4
    Kong_joint5 = Hong_joint5+pi/2


In [ ]:
def begin_rotate(carton_angle):
    carton_length = 0.303
    
    #phi, tilt angle
    alpha = 27
    
    print "finish is_descend"
    is_descend = True
    global global_speed

    tilt_fix_pose = (0, 0, 0, 0, 0, (carton_angle-pi))
    
    #tilt
    Tbf = m3d.Transform(tilt_fix_pose)
    Rbf = Tbf.orient
    Delta_f = (carton_length*(1-cos(alpha*pi/180)),0,carton_length*sin(alpha*pi/180),0,alpha*pi/180,0)
    Delta_b = m3d.Transform()
    Delta_b.pos = Rbf*m3d.Transform(Delta_f).pos
    R_delta_f = m3d.Transform(Delta_f).orient
    Delta_b.orient = Rbf*R_delta_f*Rbf.inverse
    Rot_base = m3d.Transform.get_pose_vector(Delta_b)+m3d.Transform.get_pose_vector(robHong.get_pose())
    robHong.set_pose(m3d.Transform(Rot_base), acc=0.1, vel=0.1)  # apply the new pose

    #pivot
    #beta parameter
    inverse_act_beta = m3d.Transform((0,0,0,0,68*pi/180,0))
    #gamma  parameter
    inverse_act_gamma = m3d.Transform((0,0,0,-73*pi/180,0,0))
    
    inverse_act_final = m3d.Transform((0,0,0,0,0,10*pi/180))
    
    robHong.add_pose_tool( inverse_act_beta, acc=0.1, vel=0.03*global_speed, wait=True, command="movel", threshold=None)
    robHong.add_pose_tool( inverse_act_gamma, acc=0.1, vel=0.03*global_speed, wait=True, command="movel", threshold=None)
    robHong.add_pose_tool( inverse_act_final, acc=0.1, vel=0.03*global_speed, wait=True, command="movel", threshold=None)
    rospy.sleep(0.5)

    gposition(240)
    rospy.sleep(0.5)
    robHong.movel_tool((0, 0, -0.07, 0, 0, 0), acc=0.1, vel=0.1*global_speed)
    return
    
    


In [10]:
def go_to_carton_camera_home():
    global global_speed
    
    Hong_joint0 = -0.973358456288473
    Hong_joint1 = -1.6687057654010218
    Hong_joint2 = -1.2589800993548792
    Hong_joint3 =  1.357000470161438
    Hong_joint4 = -1.5656326452838343
    Hong_joint5 = 2.1668648719787598

    robHong.movej((Hong_joint0,Hong_joint1, Hong_joint2, Hong_joint3, Hong_joint4, Hong_joint5), 1, 0.2*global_speed)


In [11]:
def go_to_blister_camera_home():
    Hong_joint0 = pi*0/180
    Hong_joint1 = pi*-68.03/180
    Hong_joint2 = pi*-137.81/180
    Hong_joint3 = pi*-64.16/180
    Hong_joint4 = pi*90.0/180
    Hong_joint5 = pi*-90.0/180

    robHong.movej((Hong_joint0,Hong_joint1, Hong_joint2, Hong_joint3, Hong_joint4, Hong_joint5), 1, 0.2)

In [12]:
def go_to_blister_camera_home2():
    blister_camera_home_pose = [0.400887521814743, 0.005921977874024514,0.07993296962552376,-3.140736877197115,-0.021274657722516237,-0.0004956758065318037]
    robHong.movel(blister_camera_home_pose, acc=0.02, vel=0.05)

In [14]:
def Kong_go_to_home():
    robKong.set_tcp((0, 0, 0.106, 0, 0, 0))
    
    Kong_joint0 = pi*66.00/180
    Kong_joint1 = pi*-76.90/180
    Kong_joint2 = pi*111.34/180
    Kong_joint3 = pi*237.00/180
    Kong_joint4 = pi*-90.0/180
    Kong_joint5 = pi*67.91/180
    
    robKong.movej((Kong_joint0,Kong_joint1, Kong_joint2, Kong_joint3, Kong_joint4, Kong_joint5), 1, 0.4)


In [15]:
def Kong_go_to_home2():
    robKong.set_tcp((0, 0, 0.106, 0, 0, 0))
    
    Kong_joint0 = pi*66.04/180
    Kong_joint1 = pi*-76.90/180
    Kong_joint2 = pi*111.27/180
    Kong_joint3 = pi*237.00/180
    Kong_joint4 = pi*-90.35/180
    Kong_joint5 = pi*78.40/180

    robKong.movej((Kong_joint0,Kong_joint1, Kong_joint2, Kong_joint3, Kong_joint4, Kong_joint5), 1, 0.2*global_speed)


In [29]:
from thin_obj_bin_picking.msg import carton_pose

def pose_callback():
    global global_speed
    global tag_pose_x
    global tag_pose_y
    global tag_angle
    
    #Kong move to home position
    Kong_go_to_home()
    rospy.sleep(1)

    #Kong move to the center point of tag
    robKong.movel((tag_pose_x-0.017 , -0.44-(tag_pose_y+0.0434)+0.015, 0, 0, 0, 0), acc=0.2, vel=0.2,relative=True)
    
    robKong.movel_tool((0, 0, 0, 0, 0, tag_angle), acc=0.2, vel=0.2)
    robKong.movel_tool((0.155, 0, 0, 0, 0, 0), acc=0.2, vel=0.1)
    rospy.sleep(1)
    robKong.movel((0, 0, -0.214, 0, 0, 0), acc=0.02, vel=0.05,relative=True)
     
    robKong.movel_tool((0, 0, 0, 0, -20.0*math.pi/90, 0), acc=0.1, vel=0.1)
    robKong.movel((0, 0, -0.002, 0, 0, 0), acc=0.1, vel=0.1, relative=True)
    
    #Hong move to tilt point
    robHong.set_tcp((0, 0, 0, 0, 0, 0))
    rospy.sleep(1)
    
    robHong.movel((0, 0, -0.20, 0, 0, 0), acc=0.2, vel=0.2*global_speed,relative=True)
    
    robHong.set_tcp((-0.04, 0.0116, 0.270, 0, 0, 0))
    rospy.sleep(1)
    
    rospy.sleep(1)
    pose = robHong.getl()
    

    print 'tag pose x', tag_pose_x, ', tag_pose y', tag_pose_y, ', tag_angle', tag_angle
    pose[0] -= tag_pose_x
    pose[1] += tag_pose_y
 
    pose[0] += 0.0118
    pose[1] += -0.2008
    
    
    robHong.movel(pose, acc=0.2, vel=0.2*global_speed)
    
    robHong.movel_tool((0, 0, 0, 0, 0, (tag_angle)), acc=0.2, vel=0.2*global_speed)
    
    robHong.movel_tool((-0.1274-0.03, -0.063, 0, 0, 0, 0), acc=0.2, vel=0.1*global_speed)

    robHong.movel_tool((0, 0, 0, 0, 0, -43*pi/180), acc=0.2, vel=0.1*global_speed)
    
    #go descend   
    robHong.movel((0, 0, -0.2995 -0.007, 0, 0, 0), acc=0.2, vel=0.1, wait=False, relative=True)
    
    #detect godown force z
    rospy.sleep(0.5)
    wrench = rospy.wait_for_message('/robotiq_ft_wrench', WrenchStamped, timeout = None)
    init_fz = wrench.wrench.force.z
    print 'init fz is', init_fz
    
    while True:
        wrench = rospy.wait_for_message('/robotiq_ft_wrench', WrenchStamped, timeout = None)
        current_fz = wrench.wrench.force.z
        delta_z = abs(current_fz - init_fz)
        if delta_z > 5:
            break;

    print 'exit go down loop'
    robHong.stopl()

    #go to contact point
    robHong.translate_tool((0.03*cos(45*pi/180),0.03*sin(45*pi/180),0),vel=0.05 * global_speed, wait=False)
    
    #detect the contact torque with object
    rospy.sleep(0.5)
    wrench = rospy.wait_for_message('/robotiq_ft_wrench', WrenchStamped, timeout = None)
    init_torque_y = wrench.wrench.torque.y
    print 'init_torque_y is', init_torque_y
    
    while True:
        wrench = rospy.wait_for_message('/robotiq_ft_wrench', WrenchStamped, timeout = None)
        current_torque_y = wrench.wrench.torque.y
        delta_ty = abs(current_torque_y - init_torque_y)
        if delta_ty > 1:
            print 'delta ty exceed'
            break;

    print 'exit go to contact loop'
    robHong.stopl()
    
    begin_rotate(tag_angle)
    
pose_topic = '/carton_pose' #"/usb_cam/image_raw"
rospy.Subscriber(pose_topic, carton_pose, pose_callback)

In [18]:
#tag detection

is_send_carton_img = 0
is_send_blister_img = 0

carton_img_count = 0
blister_img_count = 0

tag_pose_x = 0
tag_pose_y = 0
tag_angle = 0

from apriltags_ros.msg import AprilTagDetectionArray
from tf.transformations import euler_from_quaternion, quaternion_from_euler


def tag_callback(msg):
    global is_send_tag
    global tag_pose_x
    global tag_pose_y
    global tag_angle
    
    if is_send_tag == 1:
        tag_orientation = msg.detections[0].pose.pose.orientation
        rpy = euler_from_quaternion([tag_orientation.x, tag_orientation.y, tag_orientation.z, tag_orientation.w])
    #     print rpy[0] / 3.1415926 * 180, rpy[1] / 3.1415926 * 180 , rpy[2] / 3.1415926 * 180
        tag_angle = rpy[2]
        tag_pose_x = msg.detections[0].pose.pose.position.x
        tag_pose_y = msg.detections[0].pose.pose.position.y
        
        print 'tag_x = ', tag_pose_x, ',tag_y = ', tag_pose_y, ',tag angle = ', tag_angle/ 3.15159296 * 180
        is_send_tag = 0
    
    
tag_topic = '/tag_detections' 

rospy.Subscriber(tag_topic, AprilTagDetectionArray, tag_callback)

In [19]:
carton_img_count = 0
blister_img_count = 0
carton_count = 0

In [38]:
#main entry function for the program
#1. all robots go to home, and detect the april tag
global is_send_tag

try:
    gposition(100)
    rospy.sleep(1)
    jposition(120)
    
    go_to_carton_camera_home()
    Kong_go_to_home()
    rospy.sleep(1)
    is_send_tag = 1

except rospy.ROSInterruptException: pass

tag_x =  -0.0149518426538 ,tag_y =  0.026426575455 ,tag angle =  -6.17951961664


In [39]:
# after tag detection, move the robots

pose_callback()


tag pose x -0.0149518426538 , tag_pose y 0.026426575455 , tag_angle -0.108196280667
init fz is 18.6900005341
exit go down loop
init_torque_y is -0.264999985695


KeyboardInterrupt: 